# NEURAL NETWORKS AND DEEP LEARNING

---
A.A. 2021/22 (6 CFU) - Dr. Alberto Testolin, Dr. Umberto Michieli
---


# Homework 1 - Supervised Deep Learning

### Author: Michele Guadagnini - Mt.1230663

In [ ]:
# Total running time on low-end dual-core CPU: < 50 min

# Regression Task

In [ ]:
### ADDITIONAL LIBRARIES THAT NEED INSTALLATION (uncomment if needed)

#!pip install skorch

In [ ]:
# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms

# python imports
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import copy
import datetime
from scipy import stats

# additional libraries
import skorch

# seed to set random states
magic_num = 23

In [ ]:
# setting the device
if torch.cuda.is_available():
    print('GPU available')
    device = torch.device("cuda")
else:
    print('GPU not available')
    device = torch.device("cpu")
    print("Available CPU cores:", os.cpu_count())


In [ ]:
print("device set to: ", device.type)

# setting n_jobs to number of cores if running on CPU, None if on GPU
Njobs = os.cpu_count() if (device.type=="cpu") else None
### Njobs will be used as argument of the random search function

In [ ]:
# setting random state
np.random.seed(magic_num)
torch.manual_seed(magic_num)
random.seed(magic_num)
if (device.type == "cuda"): 
    torch.cuda.manual_seed(magic_num)

## Guidelines

* The goal is to train a neural network to approximate an unknown function:
$$ 
f:\mathbb{R}→\mathbb{R} \\
x↦y=f(x) \\
\text{network}(x) \approx f(x)
$$
* As training point, you only have noisy measures from the target function.
$$
\hat{y} = f(x) + noise
$$
* Consider to create a validation set from you training data, or use a k-fold cross-validation strategy. You may find useful these functions from the `scikit-learn` library:
    - [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
    - [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold) 

# SOLUTION

## Outline of the notebook

The solution of this homework can be divided as follows:

1. **Data download and organization**: data are downloaded in a local folder, then loaded in memory and shuffled. Also, the dataset object that will be used to train the network is created.

1. **Definition of the network model** that will be fitted to the data, named `FC2_Net`. The implementation will provide the possibility to set some hyper-parameters (like number of units, which activation to use, dropout, ...).

1. **Random search over the set of hyper-parameters** to identify a proper configuration for this task and dataset. The search will be done with scikit-learn function `RandomizedSearchCV` by wrapping up the PyTorch network model within the package `skorch`. Also, by setting `refit = True` inside the random search object, the found best-performing model is trained on the whole training set, providing a predictor.

1. **Cross Validation mean prediction**: once done the random search, we test also a different approach: we take the resulting best set of hyper-parameters and we train 5 of copies of this model in a 5-fold cross validation setup. Then, at the end of the training we can use the mean of their predictions as actual prediction. 

1. **Performance over test dataset**: Both the obtained predictors are then applied to the test set and the loss function (MSE) is computed to evaluate their performance.

1. **Network Analysis**: we select the best model returned from the random search and we visualize its weights distributions of each layer and also its activations profiles. Finally, we also try to change some hyper-parameters (like activation function, regularization intensity, ...) and train again the network in order to see the effects on the weights histograms and activations.

## Data download and organization

In [ ]:
# download the dataset
if not os.path.isdir("regression_dataset"):
    !wget -P regression_dataset https://gitlab.dei.unipd.it/michieli/nnld-2021-22-lab-resources/-/raw/main/homework1/train_data.csv
    !wget -P regression_dataset https://gitlab.dei.unipd.it/michieli/nnld-2021-22-lab-resources/-/raw/main/homework1/test_data.csv 

Now we load the dataset into a pandas dataframe and plot it.

In [ ]:
train_df = pd.read_csv('regression_dataset/train_data.csv')
print(f"train dataset shape: {train_df.shape}")

fig = plt.figure(figsize=(10,6))
plt.scatter(train_df["input"], train_df["label"])
plt.title("Training Points")
plt.xlabel("Input")
plt.ylabel("Label")
plt.grid()
plt.show()

fig.savefig("trainpts.pdf", bbox_inches='tight')

In [ ]:
from sklearn.utils import shuffle

# shuffling the dataset 
train_df = shuffle(train_df, random_state=magic_num)

Here we create all the tools we need to deal with the data. First of all we define the class `NpDataset` inheriting from the `torch.utils.data.Dataset` creating a *map-style* dataset object by redefining the functions `__getitem__` and `__len__`. <br>
Then we also define the transformation of data from normal array to `Tensor` object.

In [ ]:
class NpDataset(Dataset):

    def __init__(self, Arr2D, transform=None):
        """
        Args:
            Arr2D(numpy array): 2-columns array in the format [input, label].
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.transform = transform
        self.data = Arr2D

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transform is not None:
            sample = self.transform(sample)
        return sample

In [ ]:
class ToTensor(object):
    """Convert sample to Tensors."""

    def __call__(self, sample):
        x, y = sample
        return (torch.tensor([x]).float(),
                torch.tensor([y]).float())

In [ ]:
# bulding train datasets 
composed_transform = transforms.Compose([ToTensor()])
train_dataset = NpDataset(train_df.values, transform=composed_transform)

train_dataset.__len__()

## Network model definition

The cell below defines the model that will be used to fit the data, `FC2_Net`. It allows to define a fully-connected network with two hidden layers with customizable number of neurons in each layer. Also dropout probabilities after first hidden layer (`Pdrop1`) and second hidden layer (`Pdrop2`) are passed as arguments to the `__init__` function, allowing to activate the respective dropout layers. Other things customizable are: the activation function, with `ReLU` as default; the initialization scheme; also batch normalization layers after each linear layer can be added by means of a boolean flag, `batch_norm`.  <br>

For simple regression tasks, also a network with only one hidden layer could be sufficient to fit the data, but our dataset is noisy and also presents some holes in its range, hence a 2 hidden layers model has been adopted. Eventual overfitting will be mitigated with the use of regularization techniques.

In [ ]:
# network with 2 hidden layers fully connected
class FC2_Net(nn.Module):
    
    def __init__(self, Nh1, Nh2, Pdrop1=0, Pdrop2=0, Ni=1, No=1, 
                 activ=nn.ReLU(), init_scheme="default", batch_norm=False):
        """
        Nh1          - number of Neurons in first hidden layer
        Nh2          - number of Neurons in second hidden layer
        Pdrop1       - dropout probability on first layer (optional)
        Pdrop2       - dropout probability on second layer (optional)
        Ni           - Input size
        No           - Output size
        activ        - activation function to use (default is torch.nn.ReLU())
        init_scheme  - initialization scheme to use (torch.nn.init object).
                       default is Uniform(-k, k), with k = sqrt[ 1/(Ni) ]
        batch_norm   - boolean to activate batch normalization
        """
        super().__init__()
        
        # setting random state (needed to ensure reproducibility of randomized search)
        torch.manual_seed(magic_num)
        
        self.Nlyrs = 2  # number of hidden layers
        
        ### linear layers
        self.fc_h1  = nn.Linear(in_features=Ni , out_features=Nh1)   # hidden layer 1
        self.fc_h2  = nn.Linear(in_features=Nh1, out_features=Nh2)   # hidden layer 2
        self.fc_out = nn.Linear(in_features=Nh2, out_features=No )   # output layer
        
        ### dropout option
        self.drp1 = nn.Dropout(p=Pdrop1)
        self.drp2 = nn.Dropout(p=Pdrop2)
        
        ### activation function
        self.act = activ
        
        ### initialization scheme selection (optional)
        if init_scheme != "default":
            # use init_scheme to initialize weights
            init_scheme(self.fc_h1.weight)
            init_scheme(self.fc_h2.weight)
            init_scheme(self.fc_out.weight)
            
        ### batch normalization (optional)
        self.batch_norm = batch_norm
        if self.batch_norm:            
            # define batch normalization layers
            self.bn_h1 = nn.BatchNorm1d(Nh1)
            self.bn_h2 = nn.BatchNorm1d(Nh2)
            
    def forward(self, x, additional_out=False):
        
        # first layer
        x = self.fc_h1(x)
        x = self.drp1(x)     # applied only if Pdrop1 != 0
        x = self.act(x)      # activation
        if self.batch_norm:
            x = self.bn_h1(x)
        
        # second layer
        x = self.fc_h2(x)
        x = self.drp2(x)     # applied only if Pdrop2 != 0
        x = self.act(x)      # activation 
        if self.batch_norm:
            x = self.bn_h2(x)
        
        # output layer without activation
        x = self.fc_out(x)

        return x


## Randomized Search for hyper-parameters

To evaluate which is the best hyper-parameters configuration to adopt a Randomized Search procedure has been implemented. A Random Search strategy should perform better than a Grid Search one in case of a large number of hyper-parameters to optimize, especially if some hyper-parameters are more important than others. <br>
Since our dataset is very small, a good approach to evaluate the hyper-parameters is through **cross validation**. For this purpose we used the `skorch` package which allows us to wrap the *PyTorch* model inside another object (`skorch.NeuralNetRegressor`) that provides an interface compatible with `sklearn` framework. This way we can use the function `RandomizedSearchCV` from *sklearn*.

### Hyper-parameters space definition

In the cells below the hyper-parameters space is defined. For some parameters a list of possible values has been defined, while for continuous parameters it is also possible to provide a continuous distribution from the `scipy.stats` package.

In [ ]:
### number of units in each hidden layer
N_units = [160,192,224,256,288]

### list of optimizers to test
# adding momentum = 0.9 to the base class SGD 
class SGDmomentum09(torch.optim.SGD):
    def __init__(self, params, lr=0.001, momentum=0.9, dampening=0,
                 weight_decay=0, nesterov=False):
        super().__init__(params=params, lr=lr, momentum=momentum, dampening=dampening,
                 weight_decay=weight_decay, nesterov=nesterov)

optim_list = [SGDmomentum09,
              optim.Adam, 
             ]

### list of activations to test
activ_list = [nn.ReLU(),
              nn.Tanh(),
             ]

### hyper-parameters dict
# batch size is fixed to be equal to the dataset size because it is small and noisy, so subsampling 
#   would probably produce gradient instability during training

hp_list = []
for it in range(len(N_units)):
    hp = {
        "module__Nh1"            : [N_units[it]],                          # number of neurons in 1st layer
        "module__Nh2"            : [N_units[it]],                          # number of neurons in 2nd layer
        "module__activ"          : activ_list,                             # list of activations
        "module__Pdrop1"         : [0.]*2+list(np.linspace(0.,0.25,num=11)), # dropout probability of 1st layer
        "module__Pdrop2"         : [0.]*2+list(np.linspace(0.,0.25,num=11)), # dropout probability of 2nd layer
        "module__batch_norm"     : [False, True],                          # batch normalization flag   
        "max_epochs"             : [100,150,200],                          # number of epochs
        "lr"                     : stats.loguniform(1e-5, 1e-2),           # learning rates space
        "optimizer"              : optim_list,                             # set of optimizers
        "optimizer__weight_decay": [0.]+list(np.logspace(-3,-0.3,num=10)), # L2 regularization
    }
    hp_list.append(hp)

hyperparams_space = hp_list

### Random search procedure

In the cell below the actual random search is run. After resetting the random state, we define the skorch model `rand_search_net`, the score function `score_MSE` and the random search object `rand_search`. We also added a gradient clipping callback in order to solve some numerical problems with SGD and some particular configurations of hyper-parameters.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from skorch import NeuralNetRegressor
from skorch.callbacks import GradientNormClipping  # needed because, with SGD and some hyper-parameters
                                                   ### configurations, loss explodes to Inf
# setting random state
np.random.seed(magic_num)
torch.manual_seed(magic_num)
random.seed(magic_num)
if (device.type == "cuda"): 
    torch.cuda.manual_seed(magic_num)

# skorch object to be used in randomized search with cross validation
rand_search_net = NeuralNetRegressor(module      = FC2_Net,
                                     train_split = False,   # no validation (CV splits will be done by rand_search)
                                     device      = device,  # use selected device
                                     verbose     = 0,
                                     batch_size  = -1,      # single batch with all training points
                                     callbacks   = [GradientNormClipping(gradient_clip_value=1e6)],
                                    )
# scoring function
score_MSE = make_scorer(mean_squared_error, greater_is_better=False)

# number of hyper-parameters configurations to test
Niterations = 600

# random search with cross validation  
Ncv_splits  = 4
rand_search = RandomizedSearchCV(rand_search_net, 
                                 hyperparams_space, 
                                 refit   = True, 
                                 cv      = Ncv_splits,  # number of CV splits
                                 n_iter  = Niterations,
                                 scoring = score_MSE,
                                 verbose = 0, 
                                 n_jobs  = Njobs, 
                                 error_score = "raise", # raise error if an error occurs while computing score
                                 random_state = magic_num,
                                )

print("Random Search settings: ")
print("   # of hyper-parameters configurations to sample and test: {}".format(Niterations))
print("   # of cross-validation splits: {}".format(Ncv_splits))
print("Total number of fits to do: {}".format(Ncv_splits*Niterations))

In [ ]:
# transform training data into numpy arrays (skorch will take care of creating a Dataset and Dataloader objects)
XX = train_df["input"].values.reshape(-1,1).astype(np.float32)
YY = train_df["label"].values.reshape(-1,1).astype(np.float32)

# measure running time
RS_begin = time.time()

rand_search.fit(XX, YY) # run the randomized search 

RS_time = time.time() - RS_begin
print(f"Randomized Search time:", str(datetime.timedelta(seconds=RS_time)) )

Below we visualize in a pandas dataframe the results of the random search.

In [ ]:
pd.options.display.max_columns = None

rand_search_df = pd.DataFrame.from_dict(rand_search.cv_results_)

# sorting the dataframe by rank
rand_search_df = rand_search_df.sort_values(by="rank_test_score")

rand_search_df.drop(columns="params", inplace=True)
rand_search_df.columns = ["mean_fit_time","std_fit_time","mean_score_time","std_score_time",
                          "param_lr","max_epochs","Nh1","Nh2","Pdrop1","Pdrop2","activation","batch_norm",
                          "optimizer","weight_decay","split0_test_score","split1_test_score",
                          "split2_test_score","split3_test_score","mean_test_score","std_test_score",
                          "rank_test_score"
                         ]

# print best results of hyper-parameters search
rand_search_df.head(10)

From the dataframe above we can infer that `Adam` is performing better than `SGD` as 8 models in the top-10 were trained with it. In the same way `ReLU()` outperforms `Tanh` activation.

Below we report a summary of the hyper-parameters of the best-performing model and plot of training points together with the learned function from the Random Search best estimator (we set `refit = True` in the random search function, so the best estimator is re-trained over the whole dataset).

In [ ]:
print("SUMMARY OF THE NETWORK ARCHITECTURE WITH BEST PERFORMANCE:")
print("  best score:      ", rand_search.best_score_)
print("  best parameters: ")
for (key, val) in rand_search.best_params_.items():
    print("    ", key, ":", val)

In [ ]:
# representation of the learned function 
xline = torch.from_numpy(np.linspace(-5,5,200).reshape(-1,1)).float()

yline_rs = rand_search.best_estimator_.predict(xline)

fig = plt.figure(figsize=(10,6))
plt.title("Best Estimator resulting from Random Search")
plt.xlabel("Input")
plt.ylabel("Label")
plt.scatter(XX, YY, label="Train data") 
plt.plot(xline, yline_rs, c="red", label="best estimator prediction", lw=2)
plt.legend()
plt.grid()
plt.show()

## Average of Cross Validation trained networks

Now we want to build a regressor using a different approach: we use the hyper-parameters found above to train 5 models in a 5-fold cross validation setup. This allow us to build a regressor by averaging the predictions of these 5 models, making a better use of the very small dataset. <br>

To do this, we firstly define the procedure to train a single model inside the function `train_single_model`, at which we give in input the model architecture and hyper-parameters, the train and validation datasets, the maximum number of epochs and also other parameters of the callbacks, *EarlyStopping* and *Checkpoint*. Inside the function we create the callbacks objects, the skorch regressor and finally call the fit method. <br>

The second function we created is `cross_validate`, which takes care of splitting the dataset into the specified number of folds (5) and running the loop where the models are trained. It stores into two lists the trained networks and their checkpoints. <br>

The third function is `average_CV_prediction`, which simply computes the mean of the output predictions of the networks.

In [ ]:
from sklearn.model_selection import ShuffleSplit
from skorch.helper import predefined_split
from skorch.callbacks import Checkpoint, EarlyStopping

# function to train a single model, exploiting also early stopping and checkpoint callbacks
def train_single_model(net_model, hyper_params, train_dataset, val_dataset=None, 
                       max_epochs=50, verbosity=0, patience=5, iter_id=0, 
                       threshold=0.00001, dir_checkpt="checkpt"):
    ### callbacks definitions ###    
    # callback for progress bar display
    pbar_callback = ProgressBar_epochs(total_epochs=max_epochs)
    callbacks_list = [pbar_callback]
    
    if val_dataset is not None:
        # callback for early stopping
        early_stop = EarlyStopping(monitor   = "valid_loss",
                                   patience  = patience,
                                   threshold = threshold, 
                                  )
        # callback for retrieving best model w.r.t valid loss
        checkpoint = Checkpoint(monitor = "valid_loss_best",
                                dirname = dir_checkpt+str(iter_id), 
                               )
        callbacks_list += [early_stop, checkpoint]
    

    ### setting up the regressor object ###
    net = NeuralNetRegressor(module     = net_model,
                             criterion  = torch.nn.MSELoss,
                             batch_size              = -1, 
                             max_epochs              = max_epochs,
                             lr                      = hyper_params["lr"],
                             module__Nh1             = hyper_params["module__Nh1"],
                             module__Nh2             = hyper_params["module__Nh2"],
                             module__Pdrop1          = hyper_params["module__Pdrop1"],
                             module__Pdrop2          = hyper_params["module__Pdrop2"],
                             module__batch_norm      = hyper_params["module__batch_norm"],
                             module__activ           = hyper_params["module__activ"],
                             optimizer               = hyper_params["optimizer"],
                             optimizer__weight_decay = hyper_params["optimizer__weight_decay"],
                             train_split = predefined_split(val_dataset) if val_dataset is not None else False,
                             device      = device, # uses GPU if available
                             verbose     = verbosity,
                             callbacks   = callbacks_list,
                            )

    ### fitting the model ###
    _ = net.fit(train_dataset, y=None)
    
    print("  Final train loss: {}".format(net.history[-1]["train_loss"]))
    
    if val_dataset is not None:
        print("  Final validation loss: {}".format(net.history[-1]["valid_loss"]))
        return net, checkpoint
    else:
        return net


# function to train N networks with cross validation
def cross_validate(net_model, train_df, best_hp,
                   Ncv=5, max_epochs=50, rand_seed=42, verbosity=0, 
                   patience=5, threshold=0.00001, dir_checkpt="checkpt",
                  ): 
    
    # dataset splitting object for cross-validation 
    shuffle_split = ShuffleSplit(n_splits     = Ncv, 
                                 test_size    = 1./Ncv, 
                                 random_state = rand_seed,
                                )      
    estimators = []
    checkpoints = []
    iter_id = 0
    for train_ids, val_ids in shuffle_split.split(train_df["input"].values):
        print("Iteration: {}".format(iter_id))
        
        ### bulding train and validation datasets ###
        train_data = train_df.loc[train_ids].values
        val_data   = train_df.loc[val_ids].values      

        # defining the transformation to Tensor object
        composed_transform = transforms.Compose([ToTensor()])
        train_dataset = NpDataset(train_data, transform=composed_transform)
        val_dataset   = NpDataset(val_data  , transform=composed_transform)
        
        ### train the network ###
        trained_net, checkpoint = train_single_model(net_model,  
                                                     hyper_params  = best_hp,
                                                     train_dataset = train_dataset, 
                                                     val_dataset   = val_dataset,
                                                     max_epochs    = max_epochs, 
                                                     verbosity     = verbosity, 
                                                     patience      = patience,
                                                     threshold     = threshold,
                                                     dir_checkpt   = dir_checkpt,
                                                     iter_id       = iter_id,
                                                    )
        # storing the best performing model (to be used later)
        checkpoints.append(checkpoint)
        
        # storing the trained regressor
        estimators.append(trained_net)
        
        iter_id += 1
        
    return estimators, checkpoints


# function that computes the mean of the predictions of the trained nets
def average_CV_prediction(nets, input_tensor):     

    predicted = []
    for net in nets:
        predicted.append(net.predict(input_tensor))

    mean_prediction = np.mean(predicted, axis=0)
    
    # return single nets predictions and mean
    return predicted, mean_prediction

Here we defined a simple callback to display a progress bar while training.

In [ ]:
from skorch.callbacks import Callback
from tqdm.notebook import tqdm  

# class to display a progress bar of epochs during training
class ProgressBar_epochs(Callback):
    def __init__(self, total_epochs):
        self.Nep = total_epochs
        self.current_ep = 0  
        self.pbar = tqdm(total=self.Nep)
        
    def on_train_begin(self, net, **kwargs):
        self.current_ep = 0
        
    def on_train_end(self, net, **kwargs):
        self.pbar.close()
    
    def on_epoch_end(self, net, **kwargs):
        self.old_ep = self.current_ep
        self.current_ep = net.history[-1, "epoch"]
        self.pbar.update(self.current_ep - self.old_ep)

Now we are ready to reset the random state and run the training in cross validation.

In [ ]:
# setting random state
np.random.seed(magic_num)
torch.manual_seed(magic_num)
random.seed(magic_num)
if (device.type == "cuda"): 
    torch.cuda.manual_seed(magic_num)

# fitting "N_fits" regressors with CV
N_fits = 5
max_epochs = 1000

# measure running time
fit_begin = time.time() 

trained_nets, checkpts = cross_validate(FC2_Net, train_df, rand_search.best_params_, 
                                        Ncv         = N_fits, 
                                        max_epochs  = max_epochs, 
                                        rand_seed   = magic_num,
                                        patience    = 100,
                                        dir_checkpt = "model_checkpoint",
                                        verbosity   = 0,
                                       )
fit_time = time.time() - fit_begin
print(f"Cross Validation fit time:", str(datetime.timedelta(seconds=fit_time)) )

In [ ]:
# plot of the losses
fig, ax = plt.subplots(1,2, figsize=(13,6), sharey='row')

ax[0].set_title("Training Losses")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Loss value")
ax[0].grid()

ax[1].set_title("Validation Losses")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("")
ax[1].grid()

for ii, estim in enumerate(trained_nets):
    ax[0].plot(estim.history[:,"epoch"], estim.history[:,"train_loss"], label=("net_"+str(ii)))
    ax[0].set_yscale("log")
    ax[1].plot(estim.history[:,"epoch"], estim.history[:,"valid_loss"], label=("net_"+str(ii)))
    ax[1].set_yscale("log")

ax[0].legend()
ax[1].legend()
plt.tight_layout()
plt.show()

In the two cells below we reload the models from the checkpoints and we plot a representation of the learned function for each network and also the function represented by the average prediction.

In [ ]:
# retrieve best models from checkpoints
for net, cp in zip(trained_nets, checkpts):
    net.load_params(checkpoint=cp)

In [ ]:
# representation of the learned function 
xline = torch.from_numpy(np.linspace(-5,5,200).reshape(-1,1)).float()

y_predicted, y_cv_pred = average_CV_prediction(trained_nets, xline)

fig = plt.figure(figsize=(10,6))
plt.title("Fitted models")
plt.xlabel("Input")
plt.ylabel("Label")
plt.scatter(XX, YY, label="Train data") 
for ii, pp in enumerate(y_predicted):
    plt.plot(xline, pp, label=("net_"+str(ii)), lw=1., c="orange")
plt.plot(xline, y_cv_pred, c="red", label="cv_prediction", lw=2.)
plt.legend()
plt.grid()
plt.show()

### Evaluation of performances over test dataset

Below we load the test dataset and evaluate the MSE loss over it for each of the 5 trained networks, the average model and also the model produced by the random search function. Then we overlap in a plot the test points with the learned functions of both the average model and the one from random search.

In [ ]:
# test dataset loading
test_df = pd.read_csv('regression_dataset/test_data.csv')
print(f"test dataset shape: {test_df.shape}")

In [ ]:
### performance over test set

x_test      = torch.tensor(test_df["input"].values.reshape(-1,1)).float()
y_test_true = torch.tensor(test_df["label"].values.reshape(-1,1)).float()

# CV predictions, average prediction
y_test_pred, y_cv_test_pred = average_CV_prediction(trained_nets, x_test)

# predictions by random search best model
y_test_rs = rand_search.best_estimator_.predict(x_test)

# loss function
metric = nn.MSELoss()

test_loss = metric(torch.tensor(y_cv_test_pred), y_test_true)
print("Loss function value over Test dataset for average CV predictor: {}".format(test_loss))

trained_nets_losses = [metric(torch.tensor(arr), y_test_true) for arr in y_test_pred]
print("\nLoss function values over Test dataset for all the CV trained networks:")
for ii, arr in enumerate(y_test_pred):
    print("   net_{}: {}".format(ii, trained_nets_losses[ii]))
    
test_loss_rs = metric(torch.tensor(y_test_rs), y_test_true)
print("\nLoss function value over Test dataset for random search best model: {}".format(test_loss_rs))

In [ ]:
### plot of test points 
fig = plt.figure(figsize=(10,6))
plt.title("Test dataset predictions")
plt.xlabel("Input")
plt.ylabel("Label")

# test dataset
plt.scatter(x_test, y_test_true, c="limegreen", label="Test data", zorder=0)

# mean predictor function
plt.plot(xline, y_cv_pred, c="orange", label="Average CV learned function", lw=2, zorder=3)

# predictions by average over cross validation networks
plt.scatter(x_test, y_cv_test_pred, c="red", marker=".", s=60, label="Average CV predictions", zorder=4)

# best predictor resulting directly from RandomizedSearchCV function
yline_rs = rand_search.best_estimator_.predict(xline)
plt.plot(xline, yline_rs, c="deepskyblue", label="Rand. Search learned function", zorder=1)

# prediction by random search best model
plt.scatter(x_test, y_test_rs, marker=".", s=60, c="blue", label="Rand. Search predictions", zorder=2)

plt.legend(loc="lower right")
plt.grid()
plt.show()

fig.savefig("testpreds.pdf", bbox_inches='tight')

## Network analysis

The purpose of this section is to gain some insight about the trained networks. In particular we select the model from the random search to investigate its weights distribution and the activation profiles when applied to different input values.

In [ ]:
# select the best random search model for analysis 
network = rand_search.best_estimator_.module_ 

### Weights histograms

Below we plot the histograms of the weights of each layer.

In [ ]:
network

In [ ]:
def plot_weights_hist(network, log_scale=None, save=False):  
    ### retrieving network parameters
    # First layer
    h1_w = network.fc_h1.weight.data.cpu().numpy()
    h1_b = network.fc_h1.bias.data.cpu().numpy()

    # Second layer
    h2_w = network.fc_h2.weight.data.cpu().numpy()
    h2_b = network.fc_h2.bias.data.cpu().numpy()

    # Output layer
    out_w = network.fc_out.weight.data.cpu().numpy()
    out_b = network.fc_out.bias.data.cpu().numpy()

    ### histograms of the weights of the network
    fig, axs = plt.subplots(3, 1, figsize=(10,12))
    # first layer
    axs[0].hist(h1_w.flatten(), 50)
    axs[0].set_title("First layer weights")

    # second layer
    axs[1].hist(h2_w.flatten(), 50)
    axs[1].set_title("Second layer weights")

    # output layer
    axs[-1].hist(out_w.flatten(), 50)
    axs[-1].set_title('Output layer weights')
    
    if log_scale is not None: # setting log scale on specified images
        [axs[idx-1].set_yscale('log') for idx in log_scale]
    [ax.grid() for ax in axs]
    plt.tight_layout()
    plt.show()
    
    if save:
        fig.savefig("weights.pdf", bbox_inches='tight')
    
    return

In [ ]:
plot_weights_hist(network, save=True)

### Activations profiles

Here instead we use a hook function to retrieve the activations of hidden layers when an input value is forwarded through the network. Then we plot the activations obtained with different input values.

In [ ]:
network

In [ ]:
class ActivationProfiles():
    def __init__(self, network):
        self.net = network
        self.activation_func = self.net.act  #retrieving used activation function
        
        # Register hook on first and second hidden layer
        self.hook_h1 = self.net.fc_h1.register_forward_hook(self.get_activation_h1)
        self.hook_h2 = self.net.fc_h2.register_forward_hook(self.get_activation_h2)
        self.activation_h1 = 0
        self.activation_h2 = 0
    
    def get_activation_h1(self, layer, input, output):
        self.activation_h1 = self.activation_func(output)

    def get_activation_h2(self, layer, input, output):
        self.activation_h2 = self.activation_func(output)
        
    def close(self):
        self.hook_h1.remove()
        self.hook_h2.remove()


In [ ]:
def plot_activations(network, pts, save=False):
    activ_cache = ActivationProfiles(network)

    # Analyze activations on points in pts
    network.eval()
    ys, zs_1, zs_2 = [], [], []
    with torch.no_grad():
        for pi in pts:
            if network.batch_norm: ##
                ys.append( network( torch.tensor([pi]).float().unsqueeze(0).to(device) ) )
                zs_1.append(activ_cache.activation_h1.squeeze())
                zs_2.append(activ_cache.activation_h2.squeeze())
            else:
                ys.append( network( torch.tensor([pi]).float().to(device) ) )
                zs_1.append(activ_cache.activation_h1)
                zs_2.append(activ_cache.activation_h2)
                
    # remove hooks
    activ_cache.close()

    # Plot activations
    k = len(pts)
    fig, axs = plt.subplots(k, 2, figsize=(12,9), sharey=True)

    for idx, ax in enumerate(axs):
        ax[0].stem(zs_1[idx].cpu(), use_line_collection=True)
        ax[0].set_title("1st hidden layer activations for input x= %.2f" % pts[idx])
        ax[1].stem(zs_2[idx].cpu(), use_line_collection=True)
        ax[1].set_title("2nd hidden layer activations for input x= %.2f" % pts[idx])

    plt.tight_layout()
    plt.show()
    
    if save:
        fig.savefig("activations.pdf", bbox_inches='tight')
    
    return

In [ ]:
# points for which retrieve activations
pts = [-4., -2., 0.1, 2.5, 4.]

plot_activations(network, pts, save=True)

## Testing the effect of a different activation or regularization

In this section we try to change some of the hyper-parameters used in the previous trainings and see the effects on the weights distribution and activations profiles. We tested:
* `Tanh` activation in place of *ReLU*;
* `Strong L2 penalty`, setting the L2 coefficient to 0.5;
* without `Batch normalization`.

### 1) Tanh activation

In [ ]:
# setting random state
np.random.seed(magic_num)
torch.manual_seed(magic_num)
random.seed(magic_num)
if (device.type == "cuda"): 
    torch.cuda.manual_seed(magic_num)

# changing activation from ReLU to Tanh
params_tanh = rand_search.best_params_.copy()
params_tanh["module__activ"] = nn.Tanh()

# training
tanh_net = train_single_model(net_model     = FC2_Net,  
                              hyper_params  = params_tanh,
                              train_dataset = train_dataset, 
                              val_dataset   = None,
                              max_epochs    = params_tanh["max_epochs"], 
                              verbosity     = 0, 
                             )

# test loss
metric = nn.MSELoss()

y_test_tanh = tanh_net.predict(x_test)
test_loss = metric(torch.tensor(y_test_tanh), y_test_true)
print("  Loss over test dataset: {}".format(test_loss))

In [ ]:
# representation of the learned function 
xline = torch.from_numpy(np.linspace(-5,5,200).reshape(-1,1)).float()
yline_tanh = tanh_net.predict(xline)

fig = plt.figure(figsize=(12,7))
plt.xlabel("Input")
plt.ylabel("Label")
plt.scatter(test_df["input"].values , test_df["label"].values , c="limegreen", label="Test data")
plt.scatter(train_df["input"].values, train_df["label"].values, label="Train data") 
plt.plot(xline, yline_tanh, c="red", label="tanh_net prediction", lw=2)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plot_weights_hist(tanh_net.module_)

In [ ]:
plot_activations(tanh_net.module_, pts)

From the plots above we can say that `Tanh` activation produces a smoother prediction function with respect to `ReLU`, although not as good at representing the curve. In the activations profiles it can be noticed that positive and negative input values presents opposite activation patterns. Also, for inputs close to zero activations are almost never saturated.

### 2) Strong L2 regularization

In [ ]:
# setting random state
np.random.seed(magic_num)
torch.manual_seed(magic_num)
random.seed(magic_num)
if (device.type == "cuda"): 
    torch.cuda.manual_seed(magic_num)

# changing L2 penalty value
params_high_L2 = rand_search.best_params_.copy()
params_high_L2["optimizer__weight_decay"] = 0.5

# L2 regularization slows down the learning, so here we increase the number of epochs to compensate
params_high_L2["max_epochs"] = 300

# training
high_L2_net = train_single_model(net_model     = FC2_Net,  
                                 hyper_params  = params_high_L2,
                                 train_dataset = train_dataset, 
                                 val_dataset   = None,
                                 max_epochs    = params_high_L2["max_epochs"], 
                                 verbosity     = 0, 
                                )

# test loss
metric = nn.MSELoss()

y_test_high_L2 = high_L2_net.predict(x_test)
test_loss = metric(torch.tensor(y_test_high_L2), y_test_true)
print("  Loss over test dataset: {}".format(test_loss))

In [ ]:
# representation of the learned function 
xline = torch.from_numpy(np.linspace(-5,5,200).reshape(-1,1)).float()
yline_high_L2 = high_L2_net.predict(xline)

fig = plt.figure(figsize=(12,7))
plt.xlabel("Input")
plt.ylabel("Label")
plt.scatter(test_df["input"].values , test_df["label"].values , c="limegreen", label="Test data")
plt.scatter(train_df["input"].values, train_df["label"].values, label="Train data") 
plt.plot(xline, yline_high_L2, c="red", label="high_L2_net prediction", lw=2)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plot_weights_hist(high_L2_net.module_, log_scale=[2])

In [ ]:
plot_activations(high_L2_net.module_, pts)

Strong `L2 penalty` is restricting the weights to a very narrow range around 0, especially for the second layer, but not for the output one. Also activation patterns are heavily flattened towards zero.

### 3) Without Batch normalization

In [ ]:
# setting random state
np.random.seed(magic_num)
torch.manual_seed(magic_num)
random.seed(magic_num)
if (device.type == "cuda"): 
    torch.cuda.manual_seed(magic_num)

# removing batch normalization
params_NO_BatchNorm = rand_search.best_params_.copy()
params_NO_BatchNorm["module__batch_norm"] = False

# as above we increase the number of epochs
params_NO_BatchNorm["max_epochs"] = 300

# training
NO_BatchNorm_net = train_single_model(net_model     = FC2_Net,  
                                      hyper_params  = params_NO_BatchNorm,
                                      train_dataset = train_dataset, 
                                      val_dataset   = None,
                                      max_epochs    = params_NO_BatchNorm["max_epochs"], 
                                      verbosity     = 0, 
                                     )
# test loss
metric = nn.MSELoss()

y_test_NO_BN = NO_BatchNorm_net.predict(x_test)
test_loss = metric(torch.tensor(y_test_NO_BN), y_test_true)
print("  Loss over test dataset: {}".format(test_loss))

In [ ]:
# representation of the learned function 
xline = torch.from_numpy(np.linspace(-5,5,200).reshape(-1,1)).float()
yline_NO_bn = NO_BatchNorm_net.predict(xline)

fig = plt.figure(figsize=(12,7))
plt.xlabel("Input")
plt.ylabel("Label")
plt.scatter(test_df["input"].values , test_df["label"].values , c="limegreen", label="Test data")
plt.scatter(train_df["input"].values, train_df["label"].values, label="Train data") 
plt.plot(xline, yline_NO_bn, c="red", label="NO_BatchNorm_net prediction", lw=2)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plot_weights_hist(NO_BatchNorm_net.module_, log_scale=[2])

In [ ]:
plot_activations(NO_BatchNorm_net.module_, pts)

Removing batch normalization resulted in a wider distribution of weights of the second layer and also a less defined distribution for weights of output layer (with batch normalization active, the distribution was clearly bi-modal). Activation profiles instead remain quite similar to the ones of the best model.

## Conclusions

The use of a neural network as a regression model has been studied. While the provided train dataset was very noisy and small, the network has been able to well approximate the underlying function. <br>
Hyper-parameters search has been done with a randomized search in a cross validation setup, wrapping up the `PyTorch` network into a `skorch` object and using the function `RandomizedSearchCV` of the popular framework `sklearn`. <br>
The random search function already provided a good predictor, but also another approach has been followed: an ensemble of 5 networks with same hyper-parameters were trained in a cross validation setup and used to make predictions by averaging over their single predictions. It can be noticed that the test loss of the average predictions is smaller than the average of the single test losses; this confirms the chosen approach, although the difference is not so high. Indeed, even if sometimes there is a single net with a better test loss than the the average, it must be said that test points are not sampled uniformly in the range [-5,5], so an error in a region with more test points counts more than errors in less represented regions. Also, taking the mean of the predictions seems to mitigate the overfitting (knees in the function plot) that is present in some of the single models. <br>
Looking at the two regions in the train dataset where points were missing it can be noticed that the first one (in range [-3,-1]) can be acceptably approximated by both the obtained predictor, while the second one (in range [2,3]) is more difficult to learn. Note that in both cases the average model is performing better, probably due to weaker overfitting thanks to cross validation and also to the use of checkpoint.  